In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from scipy.optimize import minimize

In [2]:
np.random.seed(3)

In [3]:
tickers = [f"Ticker_{i}" for i in range(3)]
num_values = 10**6
annual_stock_price_df = pd.DataFrame(np.random.uniform(-10, 10, size=(num_values, len(tickers))), columns=tickers)

annual_stock_price_df.head(3)

,Ticker_0,Ticker_1,Ticker_2
0,1.015958,4.162956,-4.181905
1,0.216552,7.858939,7.925862
2,-7.488294,-5.855142,-8.970656


In [4]:
def create_functional_1(random_values):
    def wrapper(weights):
        weighted_annual_ratio_sum_df = random_values.mul(weights).sum(axis=1)

        weighted_expectation = weighted_annual_ratio_sum_df.mean()
        weighted_variance = weighted_annual_ratio_sum_df.var(ddof=0)
        return weighted_expectation / np.sqrt(weighted_variance)

    return wrapper


def create_functional_2(random_values):
    cov_matrix = random_values.cov(ddof=0).values
    np.fill_diagonal(cov_matrix, 0)
    
    mean_annual_ratio = random_values.mean().values
    var_annual_ratio_df = random_values.var(ddof=0).values
    
    def wrapper(weights):
        numerator = weights @ mean_annual_ratio
        denominator_var = (weights ** 2) @ var_annual_ratio_df
        denominator_cov = (weights @ cov_matrix) @ weights
        return numerator / np.sqrt(denominator_var + denominator_cov)

    return wrapper

    return wrapper

In [5]:
def constraint(x):
    return sum(x) - 1

con = {"type": "eq", "fun": constraint}
bounds = [(0, 1) for _ in range(len(tickers))]

functional_1 = create_functional_1(annual_stock_price_df)
functional_2 = create_functional_2(annual_stock_price_df)

minimization_functional_1 = lambda weights: - functional_1(weights)
minimization_functional_2 = lambda weights: - functional_2(weights)

In [6]:
def generate_random_weights(vector_size):
    weights = np.random.rand(vector_size)
    weights /= np.sum(weights)
    return weights


def random_search_for_initial_weights(functional_to_minimize, n_tries, vector_size):
    best_func_value = np.inf
    for _ in tqdm(range(n_tries)):
        weights = generate_random_weights(vector_size)
        res = minimize(functional_to_minimize, weights, constraints=con, bounds=bounds)
        if res.fun < best_func_value:
            best_func_value = res.fun

    return best_func_value

### 3 columns

In [7]:
func_1_value = random_search_for_initial_weights(minimization_functional_1, 15, vector_size=len(tickers))
print("functional_1:", func_1_value)

100%|██████████| 15/15 [00:43<00:00,  2.88s/it]

functional_1: -0.0022004586704341784


In [8]:
func_2_value = random_search_for_initial_weights(minimization_functional_2, 15, vector_size=len(tickers))
print("functional_2:", func_2_value)

100%|██████████| 15/15 [00:00<00:00, 325.32it/s]

functional_2: -0.002200448358604707


### 10 columns

In [9]:
tickers = [f"Ticker_{i}" for i in range(10)]
annual_stock_price_df = pd.DataFrame(np.random.uniform(-10, 10, size=(num_values, len(tickers))), columns=tickers)

bounds = [(0, 1) for _ in range(len(tickers))]

functional_1 = create_functional_1(annual_stock_price_df)
functional_2 = create_functional_2(annual_stock_price_df)

minimization_functional_1 = lambda weights: - functional_1(weights)
minimization_functional_2 = lambda weights: - functional_2(weights)

annual_stock_price_df.head(3)

,Ticker_0,Ticker_1,Ticker_2,Ticker_3,Ticker_4,Ticker_5,Ticker_6,Ticker_7,Ticker_8,Ticker_9
0,9.761039,-1.499094,-7.426843,1.250455,8.999567,2.438547,3.331949,-2.243402,-8.562468,2.543986
1,-7.679590,5.877662,7.727916,4.406803,3.425599,-4.451226,6.470824,-1.412254,-1.092605,-7.226664
2,6.350869,2.910797,-8.996612,-6.582924,7.182994,-3.031618,-8.114594,1.484563,-8.915023,0.666385


In [10]:
func_1_value = random_search_for_initial_weights(minimization_functional_1, 3, vector_size=len(tickers))
print("functional_1:", func_1_value)

100%|██████████| 3/3 [00:58<00:00, 19.34s/it]

functional_1: -0.0023526097156295832


In [11]:
func_2_value = random_search_for_initial_weights(minimization_functional_2, 3, vector_size=len(tickers))
print("functional_2:", func_2_value)

100%|██████████| 3/3 [00:00<00:00, 140.61it/s]

functional_2: -0.0023522932995312484
